# 4 Degree of Freedom Robot Arm

In [1]:
import sys
import numpy as np
from sympy import *

from tools import *

## Variables and settings

In [2]:
# Constants
l0 = symbols('l_0')
l2 = symbols('l_2')
l3 = symbols('l_3')
l4 = symbols('l_4')
le = symbols('l_e')


# Variables
t = symbols('t')
theta2 = Function(r"\theta_2")(t)
theta2_dot = diff(theta2, t)
theta2_ddot = diff(theta2, t, t)

theta3 = Function(r"\theta_3")(t)
theta3_dot = diff(theta3, t)
theta3_ddot = diff(theta3, t, t)

theta4 = Function(r"\theta_4")(t)
theta4_dot = diff(theta4, t)
theta4_ddot = diff(theta4, t, t)

d1 = Function('d_1')(t)
d1_dot = diff(d1, t)
d1_ddot = diff(d1, t, t)


alias = {}
# display as x dot instead of dx/dt, and ignore dependency (t)
alias.update({theta2: symbols(r"\theta_2"),
              theta2_dot: symbols(r"\dot{\theta_2}"),
              theta2_ddot: symbols(r"\ddot{\theta_2}"),
              theta3: symbols(r"\theta_3"),
              theta3_dot: symbols(r"\dot{\theta_3}"),
              theta3_ddot: symbols(r"\ddot{\theta_3}"),
              theta4: symbols(r"\theta_4"),
              theta4_dot: symbols(r"\dot{\theta_4}"),
              theta4_ddot: symbols(r"\ddot{\theta_4}"),
              d1: symbols(r"d_1"),
              d1_dot: symbols(r"\dot{d_1}"),
              d1_ddot: symbols(r"\ddot{d_1}"),})

# display cos(theta) as c\theta, sin(theta) as s\theta
alias.update({sin(theta2): symbols(r"s\theta_2"),
              cos(theta2): symbols(r"c\theta_2"),
              sin(theta3): symbols(r"s\theta_3"),
              cos(theta3): symbols(r"c\theta_3"),
              sin(theta4): symbols(r"s\theta_4"),
              cos(theta4): symbols(r"c\theta_4")})

## Forward Kinematics

In [5]:
R01, T01 = get_trans_mat(0, 0, l0+d1, 0)
symprint('T', 0, 1)
matprint(T01, alias)

R12, T12 = get_trans_mat(0, 0, 0, theta2)
symprint('T', 1, 2)
matprint(T12, alias)

R23, T23 = get_trans_mat(l2, 90, 0, theta3)
symprint('T', 2, 3)
matprint(T23, alias)

R34, T34 = get_trans_mat(0, 90, l3, theta4)
symprint('T', 3, 4)
matprint(T34, alias)

R4e, T4e = get_trans_mat(-le, 0, l4, 0)
symprint('T', 4, 'e')
matprint(T4e, alias)


^0T_1

Matrix([
[1, 0, 0,         0],
[0, 1, 0,         0],
[0, 0, 1, d_1 + l_0],
[0, 0, 0,         1]])

^1T_2

Matrix([
[c\theta_2, -s\theta_2, 0, 0],
[s\theta_2,  c\theta_2, 0, 0],
[        0,          0, 1, 0],
[        0,          0, 0, 1]])

^2T_3

Matrix([
[c\theta_3, -s\theta_3,  0, l_2],
[        0,          0, -1,   0],
[s\theta_3,  c\theta_3,  0,   0],
[        0,          0,  0,   1]])

^3T_4

Matrix([
[c\theta_4, -s\theta_4,  0,   0],
[        0,          0, -1,   0],
[s\theta_4,  c\theta_4,  0, l_3],
[        0,          0,  0,   1]])

^4T_e

Matrix([
[1, 0, 0, -l_e],
[0, 1, 0,    0],
[0, 0, 1,  l_4],
[0, 0, 0,    1]])

In [8]:
T04 = T01 * T12 * T23 * T34
T04.simplify()
symprint('T', 0, 4)
matprint(T04, alias)

T0e = T04 * T4e
T0e.simplify()
symprint('T', 0, 'e')
matprint(T0e, alias)

^0T_4

Matrix([
[ c\theta_2*c\theta_3*c\theta_4 + s\theta_2*s\theta_4, -c\theta_2*c\theta_3*s\theta_4 + c\theta_4*s\theta_2, c\theta_2*s\theta_3,  c\theta_2*l_2 + l_3*s\theta_2],
[-c\theta_2*s\theta_4 + c\theta_3*c\theta_4*s\theta_2, -c\theta_2*c\theta_4 - c\theta_3*s\theta_2*s\theta_4, s\theta_2*s\theta_3, -c\theta_2*l_3 + l_2*s\theta_2],
[                                 c\theta_4*s\theta_3,                                 -s\theta_3*s\theta_4,          -c\theta_3,                      d_1 + l_0],
[                                                   0,                                                    0,                   0,                              1]])

^0T_e

Matrix([
[ c\theta_2*c\theta_3*c\theta_4 + s\theta_2*s\theta_4, -c\theta_2*c\theta_3*s\theta_4 + c\theta_4*s\theta_2, c\theta_2*s\theta_3,   c\theta_2*l_2 + c\theta_2*l_4*s\theta_3 + l_3*s\theta_2 - l_e*(c\theta_2*c\theta_3*c\theta_4 + s\theta_2*s\theta_4)],
[-c\theta_2*s\theta_4 + c\theta_3*c\theta_4*s\theta_2, -c\theta_2*c\theta_4 - c\theta_3*s\theta_2*s\theta_4, s\theta_2*s\theta_3, -c\theta_2*l_3 + l_2*s\theta_2 + l_4*s\theta_2*s\theta_3 - l_e*(-c\theta_2*s\theta_4 + c\theta_3*c\theta_4*s\theta_2)],
[                                 c\theta_4*s\theta_3,                                 -s\theta_3*s\theta_4,          -c\theta_3,                                                                  -c\theta_3*l_4 - c\theta_4*l_e*s\theta_3 + d_1 + l_0],
[                                                   0,                                                    0,                   0,                                                                                                                